**LICENCE:**
This tutorial contains adaptations of material from [Programming Language Foundations in Agda](https://plfa.github.io/) by Phil Wadler and Wen Kokke. It is licensed under Creative Commons Attribution 4.0 International.

**SYNTAX:**
You can enter `→` by writing `->` and pressing TAB; pressing TAB again goes back to `->`. Similar useful combinations are:

| base form | alternate form |
|:---------:|:--------------:|
| ->  | → |
| \   | λ |
| neg | ¬ |
| top | ⊤ |
| bot | ⊥ |
| /\  | ∧ |
| \\/ | ∨ |
| iff | ⟺ |
| <   | ⟨ |
| >   | ⟩ |
| =<> | ≡⟨⟩ |
| qed | ∎ |
| N   | ℕ |
| Pi  | Π |
| Sigma | Σ |
| forall | ∀ |
| exists | ∃ |
| :: | ∷ |
| e | ε | 
| in | ∈ |

<!---

**PRO TRICKS:**
- You can ask Agda to provide the type of a closed expression. Just put the cursor nearby an expression and press SHIFT+TAB.
- You can ask Agda to compute the normal form of an expression enclosed in parantheses, such as `(λ x → x)` (already in normal form).

-->

Some auxiliary definitions which will be useful later:

````
module code.conn where

infixr 2 _∧_
data _∧_ (A : Set) (B : Set) : Set where
  ⟨_,_⟩ : A → B → A ∧ B
  
fst : {A B : Set} → A ∧ B → A
fst ⟨ a , _ ⟩ = a

snd : {A B : Set} → A ∧ B → B
snd ⟨ _ , b ⟩ = b

infixr 1 _∨_ 
data _∨_ (A : Set) (B : Set) : Set where
    left : A → A ∨ B
    right : B → A ∨ B

data ⊤ : Set where
  tt : ⊤
  
data ⊥ : Set where

⊥-elim : {A : Set} → ⊥ → A
⊥-elim ()

infix 3 ¬_ -- higher priority than ∨ and ∧

¬_ : Set → Set
¬ A = A → ⊥

infixr 1 _⟺_
_⟺_ : (A : Set) (B : Set) → Set
A ⟺ B = (A → B) ∧ (B → A)

-- (non-dependent) function composition
_∘_ : ∀ {a b c} {A : Set a} {B : Set b} {C : Set c} → (f : B → C) → (g : A → B) → A → C
f ∘ g = λ x → f (g x)

data ℕ : Set where
  zero : ℕ
  suc : ℕ → ℕ

{-# BUILTIN NATURAL ℕ #-}
````

# Lists

````
module code.list where
open import code.conn public
````

We define the type of lists of elements of type `A`,
denoted `A *` (note the space, which is mandatory):

````
infixr 5 _∷_
data _* (A : Set) : Set where
  ε : A *
  _∷_ : A → A * → A *
````

This is an example of *parametrised type*,
because its definition depends on the parameter `A : Set`.

## Equality on lists

We can inductively define a notion of equality on lists.

````
infix 4 _≡L_
data _≡L_ {A : Set} : A * → A * → Set where
  ε≡ε : ε ≡L ε
  ∷≡∷ : {a : A} {as bs : A *} → as ≡L bs → a ∷ as ≡L a ∷ bs
````
The two constructors above say that there are two ways of showing that two lists are equal:
either the two lists are empty (first constructor),
or the two lists are non-empty, they start with the same element,
and the tails are equal.

For example, we can prove

````
_ : 0 ∷ 1 ∷ ε ≡L 0 ∷ 1 ∷ ε
_ = ∷≡∷ (∷≡∷ ε≡ε)
````

We can easily prove some of useful properties of list equality.

### Reflexivity

The proof proceeds by induction on the list:

````
≡L-refl : {A : Set} {as : A *} → as ≡L as
≡L-refl {A} {ε} = ε≡ε
≡L-refl {A} {a ∷ as} = ∷≡∷ (≡L-refl {A} {as})
````

The base case is immediately proved by the constructor `ε≡ε : ε ≡L ε`.
In the inductive case, the inductive assumption `≡L-refl {A} {as}`
gives us a proof for `as ≡L as`,
to which we apply the constructor `∷≡∷`
and obtain a proof for `a ∷ as ≡L a ∷ as`.

### Symmetry

This time, the proof proceeds by induction on the evidence that `as ≡L bs` holds:

````
≡L-sym : {A : Set} {as bs : A *} → as ≡L bs → bs ≡L as
≡L-sym ε≡ε = ε≡ε
≡L-sym {A} (∷≡∷ {a} {as} {bs} as≡bs) = ∷≡∷ {A} {a} {bs} {as} (≡L-sym {A} {as} {bs} as≡bs)
````

The base case is trivial: if two lists are equal because of `ε≡ε`,
then they are both the empty list.
In the inductive case, we added all implicit parameters to clarify what is going on.
In fact, we could also have just written:

```agda
≡L-sym (∷≡∷ as≡bs) = ∷≡∷ (≡L-sym as≡bs)
```

Here, `as≡bs` is a proof that `as ≡L bs` holds,
by inductive assumption `≡L-sym as≡bs`
we obtain a proof that `bs ≡L as` holds (where the lists are swapped),
and thus `∷≡∷ (≡L-sym as≡bs)` is a proof that `a ∷ bs ≡L a ∷ as` holds, as required.

### Transitivity

**Exercise.** Prove that list equality is a transitive relation:

````
postulate ≡L-trans : {A : Set} {as bs cs : A *} → as ≡L bs → bs ≡L cs → as ≡L cs
````

*Hint:* Do induction on the evidence that `as ≡L bs` and `bs ≡L cs` hold.
You will need only two cases, since Agda can correctly infer that if `as` is not the empty list, then the same holds for `bs` and `cs`.

In [ ]:
module code.list.trans where
open import code.list hiding (≡L-trans)

≡L-trans : {A : Set} {as bs cs : A *} → as ≡L bs → bs ≡L cs → as ≡L cs
≡L-trans = ?

?0 : as ≡L bs → bs ≡L cs → as ≡L cs


````
module code.list-continue where
open import code.list public
````

<!--

### Congruence

List equality is a *congruence*, in the sense that applying a function to equal lists returns equal results.
A much more general notion of congruence for equality hold,
but this one is sufficient in our context:

We will not prove congruence, which in fact can be proved only in the more general context.

-->

### Equality chains

We define some notation that allows an intuitive equational reasoning.

````
infix 1 beginL_
infixr 2 _≡L⟨⟩_ _≡L⟨_⟩_
infix 3 _∎L

beginL_ : {A : Set} {x y : A *} → x ≡L y → x ≡L y
beginL x≡y = x≡y

_≡L⟨⟩_ : {A : Set} (x : A *) {y : A *} → x ≡L y → x ≡L y
x ≡L⟨⟩ x≡y = x≡y

_≡L⟨_⟩_ : {A : Set} (x : A *) {y z : A *} → x ≡L y → y ≡L z → x ≡L z
x ≡L⟨ x≡y ⟩ y≡z = ≡L-trans x≡y y≡z

_∎L : {A : Set} (x : A *) → x ≡L x
x ∎L = ≡L-refl
````

We can define standard operations on lists,
such as concatenation, map, filter, etc.

## Concatenation

````
infixr 5 _++_
_++_ : {A : Set} → A * → A * → A *
ε ++ ys = ys
(x ∷ xs) ++ ys = x ∷ (xs ++ ys)
````

An example of concatenation:

````
_ : 0 ∷ ε ++ 1 ∷ 2 ∷ ε ≡L 0 ∷ 1 ∷ 2 ∷ ε
_ = ≡L-refl
````

The definition of `_++_` says that ε is the left unit of `_++_`.
We show an example use of induction on lists and equational reasoning
by proving that `ε` is the right unit of `_++_`.

````
ε-++ : {A : Set} → (xs : A *) → xs ++ ε ≡L xs
ε-++ ε = ≡L-refl
ε-++ (x ∷ xs) =
  beginL
    (x ∷ xs) ++ ε
  ≡L⟨⟩
    x ∷ (xs ++ ε)
  ≡L⟨ ∷≡∷ (ε-++ xs) ⟩
    x ∷ xs
  ∎L
````

In the base case, we have to prove `ε ++ ε ≡L ε`,
which follows from `≡L-refl` because the l.h.s. evaluates to `ε` by the definition of `_++_`.
In the inductive case, we have to prove `(x ∷ xs) ++ ε ≡L x ∷ xs`.
The first step is justified by the fact that `(x ∷ xs) ++ ε` evaluates to `x ∷ (xs ++ ε)`
by the definition of `_++_`.
By the inductive hypothesis `ε-++ xs` we have `xs ++ ε ≡L xs`,
and thus `∷≡∷` is a proof of `x ∷ (xs ++ ε) ≡L x ∷ xs`, as required
(note here that Agda figures out that `x` is the implicit parameter of `∷≡∷`).

The second case above could also be written in just one line as

```agda
ε-++ (x ∷ xs) = ∷≡∷ (ε-++ xs)
```

### Associativity of concatenation

**Exercise.** Show that concatenation of lists is an associative operation:

```agda
++-assoc : {A : Set} → (xs ys zs : A *) → (xs ++ ys) ++ zs ≡L xs ++ (ys ++ zs)
++-assoc = ?
```

*Hint:* Do induction on `xs`. In the inductive step, you will need `∷≡∷`.

In [ ]:
module code.list.assoc where
open import code.list-continue public

++-assoc : {A : Set} → (xs ys zs : A *) → (xs ++ ys) ++ zs ≡L xs ++ (ys ++ zs)
++-assoc = ?

## Map

````
module code.list.map where
open import code.list-continue public

map : {A B : Set} → (A → B) → A * → B *
map _ ε = ε
map f (a ∷ as) = f a ∷ map f as
````

### Map and concatenation

**Exercise.** Show that `map` commutes with `_++_`, in the sense that:

```agda
map-++ : {A B : Set} (f : A → B) (xs ys : A *) → map f (xs ++ ys) ≡L map f xs ++ map f ys
map-++ = ?
```

*Hint:* Do induction on `xs`. In the inductive step, you will need `∷≡∷`.

In [ ]:
module code.list.map.comm where
open import code.list.map public

map-++ : {A B : Set} (f : A → B) (xs ys : A *) → map f (xs ++ ys) ≡L map f xs ++ map f ys
map-++ = ?

## Filter

````
module code.list.filter where
open import code.list.map public

data 𝔹 : Set where
  true : 𝔹
  false : 𝔹
  
-- equality on booleans
data _≡𝔹_ : 𝔹 → 𝔹 → Set where
  tt : true ≡𝔹 true
  ff : false ≡𝔹 false

filter : {A : Set} → (A → 𝔹) → A * → A *
filter _ ε = ε
filter p (a ∷ as) with p a
... | true = a ∷ filter p as
... | false = filter p as
````

This is an example of usage of the `with` construct,
which allows us to do further pattern matching on the result of evaluating `p a`.

### Filter and concatenation

**Exercise.** Show that filter and concatenation commute:

```agda
filter-++ : {A : Set} → (p : A → 𝔹) → (as bs : A *) → filter p (as ++ bs) ≡L filter p as ++ filter p bs
```

*Hint:* Do induction on `as`. In the inductive step,
use the `with` construct as in the definition of `filter` above.

In [ ]:
module list.filter-concat where
open import code.list.filter public

filter-++ : {A : Set} → (p : A → 𝔹) → (as bs : A *) → filter p (as ++ bs) ≡L filter p as ++ filter p bs
filter-++ = ?

### Filter and map

**Exercise.** Show that filter and map commute:

```agda
filter-map : {A B : Set} (p : B → 𝔹) (f : A → B) (as : A *) → filter p (map f as) ≡L map f (filter (p ∘ f) as)
```

*Hint:* In the inductive step,
do a case analysis on the value of `p (f a)`,
where `a` is the head of the list.

Is the following true for some choice of the predicate `q`?
Is there any assumption on `f` that makes is true?

```agda
postulate filter-map2 : {A B : Set} (p : A → 𝔹) (f : A → B) (as : A *) → map f (filter p as) ≡L filter q (map f as)
```

In [ ]:
module code.list.filter-map where
open import code.list.filter public

filter-map : {A B : Set} (p : B → 𝔹) (f : A → B) (as : A *) → filter p (map f as) ≡L map f (filter (p ∘ f) as)
filter-map = ?

## Membership

````
module code.list.member where
open import code.list.filter public
````

We define an inductive predicate on lists
representing the fact that an element appears in a list.

````
infix 4 _∈_
data _∈_ {A : Set} : A → A * → Set where
  stop : {a : A} {as : A *} → a ∈ a ∷ as
  skip : {a b : A} {as : A *} → a ∈ as → a ∈ b ∷ as
````

In the base case, `stop` provides evidence that `a` appears as the head of the list.
In the inductive case, `skip` takes as argument evidence `a∈as` that `a` appears in a list `as`, and returns evidence `skip a∈as` that `a` appears in `b ∷ as`.

For instance, we can prove

````
_ : 2 ∈ 0 ∷ 1 ∷ 2 ∷ 3 ∷ ε
_ = skip (skip stop)
````

<!-- need equality to do this one
### An alternative definition

We define the following *recursive* function mapping lists to types (!).
This is called *type-level programming*.

```agda
_inside_ : {A : Set} → A → A * → Set
_ inside ε = ⊥
a inside (b ∷ as) = a ≡ b ∨ a inside as
```agda
-->

### Membership and concatenation

**Exercise.** Prove that `∈` distributes over `++`:

````
postulate ∈-++ : {A : Set} {a : A} (as : A *) {bs : A *} → a ∈ (as ++ bs) ⟺ (a ∈ as ∨ a ∈ bs)
````

We proceed in three steps:

1. Prove

```agda
∈-++1 : {A : Set} {a : A} (as : A *) {bs : A *} → a ∈ (as ++ bs) → a ∈ as ∨ a ∈ bs
```

*Hint:* There are four cases to consider:

```agda
∈-++1 ε a∈bs = ?
∈-++1 (a ∷ as) stop = ?
∈-++1 (b ∷ as) (skip a∈as++bs) with ∈-++1 as a∈as++bs
... | left a∈as = ?
... | right a∈bs = ?
```

2. Prove

```agda
∈-++2 : {A : Set} {a : A} (as : A *) {bs : A *} → a ∈ as ∨ a ∈ bs → a ∈ (as ++ bs)
```

*Hint:* There are four cases to consider:

```agda
∈-++2 _ (left stop) = ?
∈-++2 (b ∷ as) (left (skip a∈as)) = ?
∈-++2 ε (right a∈bs) = ?
∈-++2 (b ∷ as) (right a∈bs) = ?
```

3. Put together the previous to points and prove `∈-++`.

In [ ]:
module code.list.member-concat where
open import code.list.member public hiding (∈-++)

∈-++1 : {A : Set} {a : A} (as : A *) {bs : A *} → a ∈ (as ++ bs) → a ∈ as ∨ a ∈ bs
∈-++1 ε a∈bs = ?
∈-++1 (a ∷ as) stop = ?
∈-++1 (b ∷ as) (skip a∈as++bs) with ∈-++1 as a∈as++bs
... | left a∈as = ?
... | right a∈bs = ?

∈-++2 : {A : Set} {a : A} (as : A *) {bs : A *} → a ∈ as ∨ a ∈ bs → a ∈ (as ++ bs)
∈-++2 _ (left stop) = ?
∈-++2 (b ∷ as) (left (skip a∈as)) = ?
∈-++2 ε (right a∈bs) = ?
∈-++2 (b ∷ as) (right a∈bs) = ?

∈-++ : {A : Set} {a : A} (as : A *) {bs : A *} → a ∈ (as ++ bs) ⟺ (a ∈ as ∨ a ∈ bs)
∈-++ as = ?

# Intuitionistic first-order  logic

We show how dependent types can be used to implement universal quantification (dependent function space)
and existential quantification (dependent product),
thus concluding our overview of intuitionistic logic.

## The universal quantifier

````
module code.universal where
````

In intuitionistic logic a proof of $\forall (a : A) B$ is a function $f$ mapping a proof $a$ of $A$
into a proof $f\; a$ of $B\; a$, where we can see $B$ as a family of types indexed by proofs of $A$.
The universal quantifier is implemented via the *dependent function space*

````
Π : (A : Set) → (B : A → Set) → Set
Π A B = (a : A) → B a
````

This generalises implication `A → B`, which corresponds to non-dependent function space.
In this sense, in intuitionistic logic implication is a special case of universal quantification.

Universal quantification also generalises conjunction,
since the type $B_1 \wedge B_2$ is isomorphic to $\Pi\; A\; B$
where $A = \{1, 2\}$ and $B = \{ 1 \mapsto B_1, 2 \mapsto B_2 \}$.
For this reason, sometimes `Π` is called dependent product (hence the notation).

The corresponding ∀-elimination rule is just (dependent) function application:

````
apply : {A : Set} → {B : A → Set} → Π A B → (a : A) → B a
apply f x = f x
````

In order to improve the notation for universal quantification,
we introduce the following definitions.

````
infix 0 forAll
forAll : {A : Set} → (B : A → Set) → Set
forAll {A} B = Π A B

syntax forAll (λ x → B) = ∀[ x ] B
````

The definition of `forAll` just makes the first argument `A` in `Π A B` implicit,
so we can just write `forAll B` whenever the argument `A` can be deduced from the context.

The second definition allows us to write `∀[ x ] B`
as a synonym for `forAll (λ x → B)`.
In this way, we get a suggestive notation reminiscent of logic.

### The universal quantifier, ∧, and ∨

**Exercise.**
Which of the following are intuitionistic tautologies?
Which are classic tautologies?
Which are none?
Prove the intuitionistic ones.

1. $\forall a B \wedge C \to (\forall a B) \wedge (\forall a C)$.
2. $(\forall a B) \wedge (\forall a C) \to \forall a B \wedge C$.
3. $\forall a B \vee C \to (\forall a B) \vee (\forall a C)$.
4. $ \forall a B \vee C \to (\forall a B) \vee C$, where $a$ does not occurr in $C$.
5. $(\forall a B) \vee (\forall a C) \to \forall a (B \vee C)$.

*Hint:* If you cannot easily program a solution, then most likely there is no solution.

In [ ]:
module code.universal.ex1 where
open import code.universal
open import code.conn

-- 1.
∀∧-distr : {A : Set} {B C : A → Set} → ∀[ a ] B a ∧ C a → (∀[ a ] B a) ∧ (∀[ a ] C a)
∀∧-distr = ?

-- 2.
∧∀-distr : {A : Set} {B C : A → Set} → (∀[ a ] B a) ∧ (∀[ a ] C a) → ∀[ a ] B a ∧ C a
∧∀-distr = ?

-- 3.
∀∨-distr : {A : Set} {B C : A → Set} → ∀[ a ] B a ∨ C a → (∀[ a ] B a) ∨ (∀[ a ] C a)
∀∨-distr = ?

-- 4.
∀∨-distr' : {A C : Set} {B : A → Set} → ∀[ a ] B a ∨ C → (∀[ a ] B a) ∨ C
∀∨-distr' = ?

-- 5.
∨∀-distr : {A : Set} {B C : A → Set} → (∀[ a ] B a) ∨ (∀[ a ] C a) → ∀[ a ] B a ∨ C a
∨∀-distr = ?

## The existential quantifier

````
module code.existential where
````

In intuitionistic logic,
a proof of $\exists (a : A) B$ is a pair $(a, b)$,
where $a$ is a proof of $A$
and $b$ is a proof of $B\; a$.
Like in universal quantification, we can see $B$ as a family of types indexed by proofs of $A$.
The existential quantifier is implemented with the *dependent product*:

````
data Σ (A : Set) (B : A → Set) : Set where
    ⟨_,_⟩ : (a : A) → B a → Σ A B
````
Compare this with conjunction, which corresponds to non-dependent product $A \wedge B$:

```agda
data _∧_ (A : Set) (B : Set) : Set where
  ⟨_,_⟩ : A → B → A ∧ B
```
In this sense, in intuitionistic logic existential quantification `Σ` generalises conjunction,
which justifies the name dependent product.
(This can create confusion because `Π` is sometimes called dependent product too,
since also `Π` generalises conjunction.)

Existential quantification also generalises disjunction,
since the type $B_1 \vee B_2$ is isomorphic to $\Sigma\; A\; B$
with $A = \{1, 2\}$ and $B = \{ 1 \mapsto B_1, 2 \mapsto B_2 \}$.
For this reason, `Σ` is sometimes called dependent sum.

The corresponding elimination rule is just uncurrying:

````
∃-elim : {A : Set} {B : A → Set} {C : Set} → ((a : A) → B a → C) → Σ A B → C
∃-elim a→b→c ⟨ a , b ⟩ = a→b→c a b
````

Similarly as with universal quantification,
we introduce some suggestive notation inspired from logic:

````
infix 0 thereExists
thereExists : ∀ {A : Set} (B : A → Set) → Set
thereExists {A} B = Σ A B

syntax thereExists (λ x → B) = ∃[ x ] B
````

### The existential quantifier, ∧, and ∨

**Exercise.**
Establish whether the following are intuitionistic tautologies
and prove it in Agda for the positive cases:

1. $\exists a B \vee C \to (\exists a B) \vee (\exists a C)$.
2. $(\exists a B) \vee (\exists a C) \to \exists a B \vee C$.
3. $\exists a B \wedge C \to (\exists a B) \wedge (\exists a C)$.
4. $(\exists a B) \wedge (\exists a C) \to \exists a B \wedge C$.
5. $(\exists a B) \wedge C \to \exists a B \wedge C$, where $a$ does not occurr in $C$.

In [ ]:
module code.existential.prop where
open import code.existential
open import code.conn

-- 1
∃∨-distr : {A : Set} {B C : A → Set} → ∃[ a ] B a ∨ C a → (∃[ a ] B a) ∨ (∃[ a ] C a)
∃∨-distr = ?

-- 2
∨∃-distr : {A : Set} {B C : A → Set} → (∃[ a ] B a) ∨ (∃[ a ] C a) → ∃[ a ] B a ∨ C a
∨∃-distr = ?

-- 3
∃∧-distr : {A : Set} {B C : A → Set} → ∃[ a ] B a ∧ C a → (∃[ a ] B a) ∧ (∃[ a ] C a)
∃∧-distr = ?

-- 4
∧∃-distr : {A : Set} {B C : A → Set} → (∃[ a ] B a) ∧ (∃[ a ] C a) → ∃[ a ] B a ∧ C a
∧∃-distr = ?

-- 5
∧∃-distr' : {A : Set} {B : A → Set} {C : Set} → (∃[ a ] B a) ∧ C → ∃[ a ] B a ∧ C
∧∃-distr' = ?

### Universal quantification, existential quantification, and negation

**Exercise.** Which of the following hold in intuitionistic logic? Prove it.

1. $\exists a \forall b C \to \forall b \exists a C$, where $C$ depends on $a$ and $b$.
2. $\exists a \neg B \to \neg \forall a B$, where $B$ depends on $a$.
3. $\neg \forall a B \to \exists a \neg B$, where $B$ depends on $a$.


In [ ]:
module code.universal-existential where
open import code.universal
open import code.existential
open import code.conn

-- 1
-- B cannot depend on (a : A) for the swap to be possible!
∃∀-distr : {A : Set} {B : Set} {C : A → B → Set} → ∃[ a ] ∀[ b ] C a b → ∀[ b ] ∃[ a ] C a b
∃∀-distr = ?

-- 2
¬∃→∀¬ : {A : Set} {B : A → Set} → ∃[ a ] ¬ B a → ¬ (∀[ a ] B a)
¬∃→∀¬ = ?

-- 3
¬∀→∃¬ : {A : Set} {B : A → Set} → ¬ (∀[ a ] B a) → ∃[ a ] ¬ B a
¬∀→∃¬ = ?

````
module code.nat where
open import code.conn public

--data ℕ : Set where
--  zero : ℕ
--  suc : ℕ → ℕ

-- {-# BUILTIN NATURAL ℕ #-}

infixl 5 _+_
_+_ : ℕ → ℕ → ℕ
zero + m = m
(suc n) + m = suc (n + m)

infix 4 _≡ℕ_
data _≡ℕ_ : ℕ → ℕ → Set where
  z≡z : 0 ≡ℕ 0
  s≡s : {m n : ℕ} → m ≡ℕ n → suc m ≡ℕ suc n

postulate ≡ℕ-refl : {n : ℕ} → n ≡ℕ n
postulate ≡ℕ-trans : {m n p : ℕ} → m ≡ℕ n → n ≡ℕ p → m ≡ℕ p
postulate ≡ℕ-sym : {m n : ℕ} → m ≡ℕ n → n ≡ℕ m
postulate ≡ℕ-cong : {m n : ℕ} (f : ℕ → ℕ) → m ≡ℕ n → f m ≡ℕ f n

infix  1 beginℕ_
infixr 2 _≡ℕ⟨⟩_ _≡ℕ⟨_⟩_
infix  3 _∎ℕ

beginℕ_ : {x y : ℕ} → x ≡ℕ y → x ≡ℕ y
beginℕ x≡y = x≡y

_≡ℕ⟨⟩_ : (x : ℕ) {y : ℕ} → x ≡ℕ y → x ≡ℕ y
x ≡ℕ⟨⟩ x≡y = x≡y

_≡ℕ⟨_⟩_ : (x : ℕ) {y z : ℕ} → x ≡ℕ y → y ≡ℕ z → x ≡ℕ z
x ≡ℕ⟨ x≡y ⟩ y≡z = ≡ℕ-trans x≡y y≡z

_∎ℕ : (x : ℕ) → x ≡ℕ x
x ∎ℕ = ≡ℕ-refl {x}

infix 4 _≤_
data _≤_ : ℕ → ℕ → Set where
  0≤n : {n : ℕ} → 0 ≤ n
  s≤s : {m n : ℕ} → m ≤ n → suc m ≤ suc n
````

# More on lists

## Length

````
module code.list.len where
open import code.list-continue public
open import code.nat public

length : {A : Set} → A * → ℕ
length ε = 0
length (_ ∷ xs) = suc (length xs)
````

For example:

````
_ : length (0 ∷ 1 ∷ ε) ≡ℕ 2
_ = ≡ℕ-refl
````

### Length and concatenation

**Exercise.** Show that concatenation preserves the total length of the lists.

In [ ]:
module code.list.len-app where
open import code.list.len

++-len : {A : Set} → (as bs : A *) → length (as ++ bs) ≡ℕ length as + length bs
++-len {A} as bs = ?

?0 : length (as ++ bs) ≡ℕ length as + length bs


### Map and length

**Exercise.** Show that `map` preserves the length of the list.

In [ ]:
module code.list.map-len where
open import code.list.map public
open import code.list.len public

map-len : {A B : Set} (f : A → B) (as : A *) → length as ≡ℕ length (map f as)
map-len = ?

?0 : (f : A → B) (as : A *) → length as ≡ℕ length (map f as)


## Filter

### Filter and length

**Exercise.** Show that filter cannot increase the length of the list.

In [ ]:
module code.list.filter-len where
open import code.list.filter public
open import code.list.len public

filter-len : {A : Set} → (p : A → 𝔹) → (as : A *) → length (filter p as) ≤ length as
filter-len = ?

?0 : (p : A → 𝔹) (as : A *) → length (filter p as) ≤ length as


## Reverse

````
module code.list.rev where
open import code.list-continue public
````

Below is the standard definition of a function that reverses the contents of a list.

````
reverse : {A : Set} → A * → A *
reverse ε = ε
reverse (a ∷ as) = reverse as ++ a ∷ ε
````

### Reverse and concatenation

**Exercise.** Show the following relationship between inverse and concatenation.

```agda
rev-++ : {A : Set} (as bs : A *) → reverse (as ++ bs) ≡L reverse bs ++ reverse as
```

*Hint*: Do induction on `as`. In the base case,
you will need the following fact proved earlier:

```agda
ε-++ : {A : Set} → (xs : A *) → xs ++ ε ≡L xs
```

In the inductive case, you will need associativity of list concatenation and congruence:

````
postulate ++-assoc : {A : Set} → (xs ys zs : A *) → (xs ++ ys) ++ zs ≡L xs ++ (ys ++ zs)
postulate ≡L-cong : {A B : Set} (f : A * → B *) {as bs : A *} → as ≡L bs → f as ≡L f bs
````

In [28]:
module code.list.rev-concat where
open import code.list.rev public

rev-++ : {A : Set} (as bs : A *) → reverse (as ++ bs) ≡L reverse bs ++ reverse as
rev-++ = ?

?0 : (as bs : A *) → reverse (as ++ bs) ≡L reverse bs ++ reverse as


**Exercise.** As an application of `rev-++`,
show that the reverse operation is *involutive*, in the following sense:

```agda
rev-inv : {A : Set} → as ≡L reverse (reverse as)
```

*Hint:* In the inductive step, you need to apply `rev-++` before you can apply the inductive hypothesis.

In [ ]:
module code.list.rev-inv where
open import code.list.rev public

postulate rev-++ : {A : Set} (as bs : A *) → reverse (as ++ bs) ≡L reverse bs ++ reverse as

rev-inv : {A : Set} (as : A *) → reverse (reverse as) ≡L as 
rev-inv = ?

?0 : (as : A *) → reverse (reverse as) ≡L as


### Reverse and length

**Exercise.** Show that `reverse` preserves the length of the list:

```agda
rev-len : {A : Set} (as : A *) → length (reverse as) ≡ℕ length as
rev-len = ?
```

### Reverse and map

**Exercise.** Show that `reverse` and `map` commute:

```agda
rev-map : {A B : Set} (f : A → B) (as : A *) → reverse (map f as) ≡L map f (reverse as)
rev-map = ?
```

### Reverse and filter

**Exercise.** Show that `reverse` and `filter` commute:

```agda
rev-filter : {A : Set} (p : A → 𝔹) (as : A *) → reverse (filter p as) ≡L filter p (reverse as)
rev-filter = ?
```

## Fast reverse

````
module code.list.fastrev where
open import code.list.rev public
````

The function `reverse` presented above has quadratic time complexity.
A well-known more efficient way of reversing a list uses an extra parameter,
called *accumulator*:

````
reverseAcc : {A : Set} → A * → A * → A *
reverseAcc as ε = as
reverseAcc as (x ∷ xs) = reverseAcc (x ∷ as) xs

fastReverse : {A : Set} → A * → A *
fastReverse = reverseAcc ε
````

**Exercise.** We want to prove that `reverse` and `fastReverse` are equivalent.

1. First establish a useful lemma about `reverse`:

```agda
reverse-lemma : {A : Set} → (xs ys : A *) → (x : A) → reverse (x ∷ xs) ++ ys ≡L reverse xs ++ x ∷ ys
```

*Hint:* You don't need induction, but you will need `++-assoc`.

2. Then establish the following invariant satisfied by the helper function `reverseAcc`:

```agda
invariant : {A : Set} → (xs : A *) → (as : A *) → reverse xs ++ as ≡L reverseAcc as xs
```

*Hint:* Do induction on `xs`. In the inductive case,
you will need `reverse-lemma` before you can apply the inductive hypothesis.

3. Finally, 

```agda
equivalence : {A : Set} (xs : A *) → reverse xs ≡L fastReverse xs
```

*Hint:* You do not need induction, just `ε-++` and `invariant` established above.

In [31]:
module code.list.fastrev.correct where
open import code.list.fastrev public

reverse-lemma : {A : Set} → (xs ys : A *) → (x : A) → reverse (x ∷ xs) ++ ys ≡L reverse xs ++ x ∷ ys
reverse-lemma = ?

invariant : {A : Set} → (xs : A *) → (as : A *) → reverse xs ++ as ≡L reverseAcc as xs
invariant = ?

equivalence : {A : Set} (xs : A *) → reverse xs ≡L fastReverse xs
equivalence xs = ?

?0
  : (xs ys : A *) (x : A) →
    reverse (x ∷ xs) ++ ys ≡L reverse xs ++ x ∷ ys
?1 : (xs : A *) (as : A *) → reverse xs ++ as ≡L reverseAcc as xs
?2 : reverse xs ≡L fastReverse xs


## Membership revisited

````
module code.list.member2 where
open import code.list.member public
open import code.list.len public
open import code.nat public
open import code.existential public
open import code.list.rev public
````

### Membership and length

````
postulate ∈-len : {A : Set} {as : A *} → 1 ≤ length as ⟺ (∃[ a ] a ∈ as)
````

### Membership and map

````
postulate ∈-map : {A B : Set} {as : A *} {a : A} (f : A → B) → a ∈ as → f a ∈ map f as
````

When is the converse direction true?

### Membership and filter

````
postulate ∈-filter : {A : Set} {as : A *} {a : A} (p : A → 𝔹) → p a ≡𝔹 true ⟺ a ∈ filter p as
````

### Membership and reverse

````
postulate ∈-rev : {A : Set} {as : A *} {a : A} → a ∈ as ⟺ a ∈ reverse as
````

## All

````
module code.list.all where
open import code.list.member public
open import code.conn public
open import code.universal public
open import code.list.rev public
````

````
data All {A : Set} (P : A → Set) : A * → Set where
  all-ε : All P ε
  all-∷ : {a : A} {as : A *} → P a → All P as → All P (a ∷ as)
````

### An alternative definition

We define the following recursive function mapping lists to types.
This technique is called *type-level programming*.

````
All-∧ : {A : Set} → (A → Set) → A * → Set
All-∧ _ ε = ⊤
All-∧ P (a ∷ as) = P a ∧ All-∧ P as

const : {A B : Set} → B → A → B
const b _ = b

All≡All-∧ : {A : Set} → (P : A → Set) → (as : A *) → All P as ⟺ All-∧ P as
All≡All-∧ P ε = ⟨ const tt , const all-ε ⟩
All≡All-∧ P (a ∷ as) = ⟨ f , g ⟩ where
  f : All P (a ∷ as) → All-∧ P (a ∷ as)
  f (all-∷ Pa Pas) = ⟨ Pa , fst (All≡All-∧ P as) Pas ⟩
  g : All-∧ P (a ∷ as) → All P (a ∷ as)
  g ⟨ Pa , Pas ⟩ = all-∷ Pa (snd (All≡All-∧ P as) Pas)
````

### All and concatenation

````
postulate All-++ : {A : Set} (P : A → Set) (as bs : A *) → All P (as ++ bs) ⟺ All P as ∧ All P bs
````

### All and map

````
postulate All-map : {A B : Set} (P : B → Set) (f : A → B) (as : A *) → All P (map f as) ⟺ All (P ∘ f) as
````

### All and filter

````
postulate All-filter1 : {A : Set} (P : A → Set) (p : A → 𝔹) (as : A *) → All P as → All P (filter p as)

postulate All-filter2 : {A : Set} (P : A → Set) (p : A → 𝔹) (as : A *) → All P (filter p as) → All (λ a → p a ≡𝔹 true → P a) as
````

### All and reverse

````
postulate All-reverse : {A : Set} (P : A → Set) (as : A *) → All P (reverse as) ⟺ All P as
````

### All and membership

````
postulate All-∈ : {A : Set} {P : A → Set} (as : A *) → All P as ⟺ (∀[ a ] (a ∈ as → P a))
````

## Some

````
module code.list.some where
open import code.list.member public
open import code.conn public
open import code.existential public
open import code.nat public
open import code.list.rev public
````

````
data Some {A : Set} (P : A → Set) : A * → Set where
  found : {a : A} {as : A *} → P a → Some P (a ∷ as) 
  skip : {a : A} {as : A *} → Some P as → Some P (a ∷ as)
  
Some-∨ : {A : Set} → (A → Set) → A * → Set
Some-∨ _ ε = ⊥
Some-∨ P (a ∷ as) = P a ∨ Some-∨ P as

const : {A B : Set} → B → A → B
const b _ = b

Some≡Some-∨ : {A : Set} → (P : A → Set) → (as : A *) → Some P as ⟺ Some-∨ P as
Some≡Some-∨ P ε = ⟨ f , (λ x → ⊥-elim x) ⟩ where
  f : Some P ε → Some-∨ P ε
  f ()
Some≡Some-∨ P (a ∷ as) = ⟨ f , g ⟩ where
  f : Some P (a ∷ as) → Some-∨ P (a ∷ as)
  f (found Pa) = left Pa
  f (skip Pas) = right (fst (Some≡Some-∨ P as) Pas)
  g : Some-∨ P (a ∷ as) → Some P (a ∷ as)
  g (left Pa) = found Pa
  g (right Pas) = skip (snd (Some≡Some-∨ P as) Pas)
````

### Some and concatenation

````
postulate Some-++ : {A : Set} (P : A → Set) (as bs : A *) → Some P (as ++ bs) ⟺ Some P as ∨ Some P bs
````

### Some and map

````
postulate Some-map : {A B : Set} (P : B → Set) (f : A → B) (as : A *) → Some P (map f as) ⟺ Some (P ∘ f) as
````

### Some and filter

````
postulate Some-filter : {A : Set} (P : A → Set) (p : A → 𝔹) (as : A *) → Some P (filter p as) ⟺ Some (λ a → P a ∧ p a ≡𝔹 true) as
````

### Some and reverse

````
postulate Some-reverse : {A : Set} (P : A → Set) (as : A *) → Some P (reverse as) ⟺ Some P as
````

### Some and membership

````
postulate Some-∈ : {A : Set} {P : A → Set} (as : A *) → Some P as ⟺ (∃[ a ] a ∈ as ∧ P a)
````